<a href="https://colab.research.google.com" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
%load_ext autoreload
%autoreload 2
%matplotlib inline

In [2]:
!git clone https://github.com/wellbeingyang/sast2023-cv
!pip install lightning==2.0.5 transformers diffusers einops torchvision numpy matplotlib imageio scikit-image kornia

^C


Cloning into 'sast2023-cv'...
fatal: unable to access 'https://github.com/wellbeingyang/sast2023-cv/': OpenSSL SSL_read: Connection was reset, errno 10054


  Using cached lightning-2.0.5-py3-none-any.whl (1.9 MB)
  Using cached diffusers-0.19.3-py3-none-any.whl (1.3 MB)
  Using cached einops-0.6.1-py3-none-any.whl (42 kB)
  Using cached torchvision-0.15.2-cp311-cp311-win_amd64.whl (1.2 MB)
  Using cached kornia-0.6.12-py2.py3-none-any.whl (653 kB)
  Using cached backoff-2.2.1-py3-none-any.whl (15 kB)
  Using cached croniter-1.4.1-py2.py3-none-any.whl (19 kB)
  Using cached dateutils-0.6.12-py2.py3-none-any.whl (5.7 kB)
  Using cached deepdiff-6.3.1-py3-none-any.whl (70 kB)
                                              0.0/65.8 kB ? eta -:--:--
     ------                                   10.2/65.8 kB ? eta -:--:--
     ------                                   10.2/65.8 kB ? eta -:--:--
     -----------------------                41.0/65.8 kB 326.8 kB/s eta 0:00:01
     -------------------------------------- 65.8/65.8 kB 444.1 kB/s eta 0:00:00
  Using cached inquirer-3.1.3-py3-none-any.whl (18 kB)
  Using cached lightning_cloud-0.5.37-py3

# Latent Diffusion Training

In this notebook, we will train a simple `LatentDiffusion` model.

The training should take up to 20 hours for reasonable results.

Ideally, you will download this dataset once and store it as `./afhq`. If you're running on colab, it's a good idea to download it once to your personal machine (it's only 240 MB) and then upload it to your colab space when you start a new notebook.

In [ ]:
import sys
sys.path.append('./sast2023-cv/')
sys.argv = ['ipykernel_launcher.py']

import os, sys, argparse
import torch
from torch.utils.data import Dataset, DataLoader
import pytorch_lightning as pl
from pytorch_lightning import seed_everything

import matplotlib as mpl
import matplotlib.pyplot as plt
import imageio

from src import *

mpl.rcParams['figure.figsize'] = (8, 8)

In [ ]:
# Note: Actually it's totally unnecessay and silly to use argparse in jupyter notebook. 
# But anyway, it's a good chance to do some practice and it will definitey be useful someday.
parser = argparse.ArgumentParser()
parser.add_argument("--image_size", type=int, default=256, help="Image size")
parser.add_argument("--train_dataset", type=str, default="./afhq/", help="The path to your training dataset")
parser.add_argument("--device", type=str, default=0 if torch.cuda.is_available() else "cpu", help="Device number.")
parser.add_argument("--num_workers", type=int, default=0, help="Spawn how many processes to load data.")
parser.add_argument("--seed", type=int, default=42, help='manual seed')
parser.add_argument("--max_epochs", type=int, default=1000, help="Max epoch number to run.")
parser.add_argument("--ckpt_path", type=str, default="", help="Checkpoint path to load.")
parser.add_argument("--save_path", type=str, default="./ckpt/", help="Checkpoint path to save.")
parser.add_argument("--save_freq", type=int, default=1, help="Save model every how many epochs.")
parser.add_argument("--ddim_steps", type=int, default=50, help="DDIM timesteps")
# TODO begin: Add arguments lr and batch_size. It's recommended to set default lr to 1e-4 and default batch_size to 8.
parser.add_argument("--lr", type=float, default=1e-4, help="Learning rate.")
parser.add_argument("--batch_size", type=int, default=8, help="The batch size.")
# TODO end
args = parser.parse_args()
seed_everything(args.seed)

### Prepare dataset and dataloader

In [ ]:
from kornia.utils import image_to_tensor
import kornia.augmentation as KA

class SimpleImageDataset(Dataset):
    """Dataset returning images in a folder."""

    def __init__(self,
                 root_dir,
                 transforms = None):

        self.root_dir = root_dir
        self.transforms = transforms

        # set up transforms
        if self.transforms is not None:
            data_keys = ['input']

            self.input_T = KA.container.AugmentationSequential(
                *self.transforms,
                data_keys = data_keys,
                same_on_batch = False
            )

        # TODO begin: Define the image paths filtered by the `supported_formats` in your datasets
        # Hint: os.listdir
        # Challenge: Can you complete this task in one line? (hint: Python comprehension, refer to Python basics handout by Yifan Li)
        supported_formats = ["jpg", "png"]
        self.image_names = [path for path in os.listdir(args.train_dataset) if path[-3:] == any(supported_formats)]
        # TODO end

    def __len__(self):
        # TODO begin: Return the length of your dataset
        return len(self.image_names)
        # TODO end

    def __getitem__(self, idx):
        if torch.is_tensor(idx):
            idx = idx.tolist()

        img_name = os.path.join(self.root_dir, self.image_names[idx])
        image = image_to_tensor(imageio.imread(img_name)) / 255

        if self.transforms is not None:
            image = self.input_T(image)[0]

        return image

In [ ]:
import torchvision.transforms as T

CROP_SIZE = args.image_size

transform = [
    KA.RandomCrop((2 * CROP_SIZE,2 * CROP_SIZE)),
    KA.Resize((CROP_SIZE, CROP_SIZE), antialias=True),
    KA.RandomVerticalFlip()
  ]

train_dataset = SimpleImageDataset(args.train_dataset, transforms = transform)

In [ ]:
# TODO begin: Define the training dataloader using torch.utils.data.DataLoader
# Hint: check the API of torch.utils.data.DataLoader, especially arguments like batch_size, shuffle, num_workers
train_dataloader = DataLoader(
    dataset=train_dataset,
    batch_size=args.batch_size,
    shuffle=True,
    num_workers=args.num_workers
    )
# TODO end

### Model Training

In [ ]:
from src import *

# TODO begin: complete the LatentDiffusion Model in `src`
model = LatentDiffusion(lr = args.lr, batch_size = args.batch_size)
# TODO end

...but first, let's check if the used `AutoEncoder` (`model.vae`) can reconstruct our samples well.

**You should always test your autoencoder in this way when using latent diffusion models on a new dataset.**

In [ ]:
img = train_dataset[0]

# TODO begin: Show the example img and use vae to reconstruct it using matplotlib
# Hint: plt.imshow
# Challenge: What's the image shape here? Should you permute or unsqueeze it?
plt.figure()
plt.imshow
plt.subplot(1,2,1)
# Plot the original img here

plt.title('Input')

plt.subplot(1,2,2)
# Plot the reconstructed img by `mode.vae` here

plt.title('AutoEncoder Reconstruction')
# TODO end

In [ ]:
# Define the trainer using PyTorch Lightning
from lightning.pytorch.callbacks import ModelCheckpoint

checkpoint_callback = ModelCheckpoint(dirpath=args.save_path, every_n_epochs=args.save_freq)

# TODO: You can specify other parameters here, like accelerator, devices...
# You can check the pl.Trainer API here: https://lightning.ai/docs/pytorch/stable/common/trainer.html
trainer = pl.Trainer(
    max_epochs = args.max_epochs,
    callbacks = [EMA(0.9999), checkpoint_callback]
)

In [ ]:
# Easy to train the model in PyTorch Lightning in one line
trainer.fit(model, train_dataloaders=train_dataloader, ckpt_path=args.ckpt_path if args.ckpt_path else None)
# TODO Challenge: Can you add some logging and visualization codes to better babysitting the training process? 
# Hint: There are many librarys you can use, e.g. logging, tensorboard, wandb... And the easiest way: print the loss every step.

Go to sleep now ~ This one line might run for days...

Wait! Please make sure that you have save the checkpoints correctly. 

If the code breaks for some reason, you can load the checkpoint and continue training.

### Now sample images from your model!

In [ ]:
model.to(args.device)
out = model(batch_size = args.batch_size, shape = (64,64), verbose = True)
# You can also try `sampler=DDIM_Sampler(num_steps=args.ddim_steps)`, which can sample images in less than 50 steps.


In [ ]:
for idx in range(out.shape[0]):
    plt.subplot(1,len(out),idx+1)
    plt.imshow(out[idx].detach().cpu().permute(1,2,0))
    plt.axis('off')